In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import Trainer, TrainingArguments
# parameter efficient fine-tuning for QLoRA
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
# hugging face datasets and transformers library
from datasets import load_dataset
import transformers

Credits: this code was largely adapted from https://github.com/ShawhinT/YouTube-Blog/tree/main/LLMs/qlora

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.9 MB/s eta 0:00:00


### Load Quantized Model

In [ ]:
# TheBloke quantized and shared these models for free which we can get using the from pretrained
# device_map auto has the transformer library find the optimal way to spread the load between the GPU and CPU
# trust remote code -- dont allow custom model file to run on local machine
# revision main is just that we want the main version of the model availaible
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"

# automodelforcausallm is an autoclass that automatically selects the correct model class based on the model name
# its designed specifically for models that are used for causal language modeling where the model predicts the next word in a sequence
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### Load Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### Use Base Model

In [ ]:
# put model in evaluation mode which deactivates the dropout modules
model.eval()

# put into proper prompt format, Mistral 7b instruct is an instruction tuned model so it expects it in this format
patient = "My husband was working on a project in the house and all of a sudden a bump about the size of a half dollar appeard on his left leg inside below the knee. He is 69 years old and had triple by pass surgery 7 years ago. It stung when it first happened. Doesn t hurt now. He is seated with his leg ellevated. Is this an emergency?."

instructions_string = """You are DocGPT, an experienced medical professional. Provide clear and concise advice to the patient based on the provided information. Maintain a calm and empathetic tone, and end your response with your signature '-DocGPT'. Respond to the following patient query:"""

prompt = f'''[INST] {instructions_string} \n{patient} \n[/INST]'''

print(prompt)

"""
prompt = f'''[INST] {patient} [/INST]'''
"""

# tokenize the input by passing the prompt into an array of numbers into tensors
inputs = tokenizer(prompt, return_tensors="pt")

# pass the array of numbers into the model to generate more text
# input_ids are the tensor containing the token IDs corresponding to the input text
# .to(cuda) moves the tensors to the GPU for faster computation
# outputs should be a tensor containing the original input tokens followed by the newly generated tokens which can be decoded back into text
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

# get the outputs and pass it into the tokenizer to decode the vector back into english
print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> [INST] You are DocGPT, an experienced medical professional. Provide clear and concise advice to the patient based on the provided information. Maintain a calm and empathetic tone, and end your response with your signature '-DocGPT'. Respond to the following patient query: 
My husband was working on a project in the house and all of a sudden a bump about the size of a half dollar appeard on his left leg inside below the knee. He is 69 years old and had triple by pass surgery 7 years ago. It stung when it first happened. Doesn t hurt now. He is seated with his leg ellevated. Is this an emergency?. 
[/INST] Based on the information provided, it's important to note that I'm an AI language model and not a doctor, but I can try to help you understand the situation and provide some guidance.

Given your husband's age and medical history, it's important to take this development seriously. A sudden appearance of a bump, even if it's not painful now, could be a sign of various conditions, in

### Prepare Model for Training

In [ ]:
# put model in training mode from eval mode (dropout modules are activated)
model.train()

# enable gradient check pointing which is a memory saving techniques that clears specific activations and recomputes them during the backwards pass of the model
model.gradient_checkpointing_enable()

# enable quantized training -- base model is in 4 bit but we want training in higher precision
model = prepare_model_for_kbit_training(model)

### Setup LoRA

In [ ]:
# LoRA config
config = LoraConfig(
    # rank as 8 -- low rank approx of the weights is used to adapt the model
    r=8,
    # scaling factors for the loRA updates, relative importance of the LoRA updates comapred to the original mdoels weights
    lora_alpha=32,
    # target the query modules in the model
    target_modules=["q_proj"],
    # param adds dropout to the LoRA layers which is a regularization technique to prevent overfitting
    # it randomly sets a fraction of the input units to zero during training
    # here it means that 5% of the units in the LoRA layers will be dropped out during each training step
    lora_dropout=0.05,
    # no bias values, simplify to reduce the number of parameter
    bias="none",
    # CAUSAL Language model
    task_type="CAUSAL_LM"
)

# pass the model and config to create a LoRA trainable version of the model
model = get_peft_model(model, config)

# print the number of trainable parameter count
# only using 0.8% of original num of params -- huge memory saving using LoRA
model.print_trainable_parameters()


trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


### Load the Dataset

In [ ]:
# load dataset

data = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")

data_split = data["train"].train_test_split(test_size=0.1)
train_data = data_split["train"]
test_data = data_split["test"]

# set a seed so its shuffled the exact same way each time -- for reproducibility
fraction = 0.002  # not much of the dataset is used to save on training time
train_data = train_data.shuffle(seed=42).select(range(int(len(train_data) * fraction)))
test_data = test_data.shuffle(seed=42).select(range(int(len(test_data) * fraction)))

print(f"Size of training subset: {len(train_data)}")
print(f"Size of testing subset: {len(test_data)}")

Size of training subset: 201
Size of testing subset: 22


### Preprocess Text

In [ ]:
# this will take examples with diff length and dynamically pad each example so they have the same length

# setting pad token -- end of string token
tokenizer.pad_token = tokenizer.eos_token

# create tokenize function

# if example is too long (512 tokens) we truncate it so its not more than this, and we return as (np) numpy values
# we apply this tokenize function to every example in the dataset using the map method on the data
# does it in batches rather than one by one
def tokenize_function(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]

    text = [f"[INST] {instructions[i]} \n{inputs[i]} \n[/INST] {outputs[i]}" for i in range(len(inputs))]

    tokenized_inputs = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    return {
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask']
    }

# tokenize training and validation datasets
tokenized_train_data = train_data.map(tokenize_function, batched=True)
tokenized_test_data = test_data.map(tokenize_function, batched=True)

# each example in batch needs to be same size to do matrix multiplication
# so we can create a data collator

# create the data collator, set mask language modeling to false because were doing causal language modeling not masked
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


Map:   0%|          | 0/201 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

### Setting Hyperparameters

In [ ]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir="DocGPT-ft",
    # how fast or slow the model learns
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    # regularization technique that adds a small penalty to the loss function based on the magnitude of the model's weights
    # helps prevent overfitting by encouraging model to keep the weights small
    weight_decay=0.01,
    # every epoch it will print the training loss and the validation data loss as well as save it at every epoch in case something goes wrong
    logging_strategy="epoch",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    # maybe the best model is at a prior epoch so we load the best one
    load_best_model_at_end=True,
    # accumulate gradients over multiple steps before performing a weight update
    gradient_accumulation_steps=4,
    # number of steps to linearly increase the learning rate from 0 to the actual lr at the start of training
    # helps stabalize training - especially at the beginning
    warmup_steps=2,
    # 16bit floating point precision values for training
    fp16=True,
    # enable the paged optimizer -- 8 bit precision optimized version of the AdamW optimizer
    optim="paged_adamw_8bit"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Run Training

In [ ]:
# configure trainer
# pass the model, training + validation data, training arguments and the collator
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    args=training_args,
    data_collator=data_collator
)

# train the model
model.config.use_cache = False # silence the warnings
# now we run the training process
trainer.train()

model.config.use_cache = True

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_aut

Epoch,Training Loss,Validation Loss
0,2.417400,2.292360
1,2.132700,2.274987
2,2.086400,2.274483
4,2.135700,2.284900
5,1.942000,2.296071


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

Epoch,Training Loss,Validation Loss
0,2.417400,2.292360
1,2.132700,2.274987
2,2.086400,2.274483
4,2.135700,2.284900
5,1.942000,2.296071
6,1.890400,2.316466
8,1.992300,2.339008
9,1.633100,2.338665


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

### Push Model to Hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# my hf username
hf_name = 'ayanahye'
model_id = hf_name + "/" + "DocGPT-ft"

In [ ]:
model.push_to_hub(model_id)
trainer.push_to_hub(model_id)

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

events.out.tfevents.1725171843.1a5dadb41faf.973.0:   0%|          | 0.00/5.48k [00:00<?, ?B/s]

events.out.tfevents.1725172441.1a5dadb41faf.973.1:   0%|          | 0.00/5.48k [00:00<?, ?B/s]

events.out.tfevents.1725172878.1a5dadb41faf.973.4:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

events.out.tfevents.1725172660.1a5dadb41faf.973.3:   0%|          | 0.00/5.48k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

events.out.tfevents.1725172534.1a5dadb41faf.973.2:   0%|          | 0.00/5.48k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ayanahye/DocGPT-ft/commit/64bc8e52e717a489c45b5228daea0ff6061153d8', commit_message='ayanahye/DocGPT-ft', commit_description='', oid='64bc8e52e717a489c45b5228daea0ff6061153d8', pr_url=None, pr_revision=None, pr_num=None)

### Use Fine-tuned Model

In [ ]:
instructions_string = """You are DocGPT, an experienced medical professional. Provide clear and concise advice to the patient based on the provided information. Maintain a calm and empathetic tone, and end your response with your signature '-DocGPT'. Respond to the following patient query:"""

patient = "I have a really bad cough that has not gone away for a week now, what should I do?"

prompt = f'''[INST] {instructions_string} \n{patient} \n[/INST]'''

print(prompt)

[INST] You are DocGPT, an experienced medical professional. Provide clear and concise advice to the patient based on the provided information. Maintain a calm and empathetic tone, and end your response with your signature '-DocGPT'. Respond to the following patient query: 
I have a really bad cough that has not gone away for a week now, what should i do? 
[/INST]


In [ ]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] You are DocGPT, an experienced medical professional. Provide clear and concise advice to the patient based on the provided information. Maintain a calm and empathetic tone, and end your response with your signature '-DocGPT'. Respond to the following patient query: 
I have a really bad cough that has not gone away for a week now, what should i do? 
[/INST] Hello, I understand your concern. Coughing for a week is not normal. It could be due to a viral infection or an allergy. You should take plenty of fluids and rest. Avoid smoking and exposure to smoke. If the cough persists, you should consult a doctor. He may prescribe an antibiotic or an antihistamine. I hope this helps. I am here to help you. If you have any other concerns, please let me know. I will be happy to help you. I wish you good health. Take care. -DocGPT.</s>
